<center> <h1> <span style="color:black"> IA|BE Data Science Certificate - Module 3 - Computer lab 2  </h1> </center>



<center> <h2> <span style="color:red"> Building Data Visualization Applications with Plotly & Dash</h1> </center>

# Introduction

Business Intelligence (BI) dashboards are a very neat tool to extract insights from your data. This is almost always a good sanity check before jumping into the modeling part, both to make sure that you understand your data and to validate that there are no unusual things hidden. This workshop intends to show how you can create your own custom BI tool in Python from scratch using Plotly & Dash.

[Plotly](https://plotly.com/graphing-libraries/) is an open-source graphing library to make interactive charts and maps for Python, R, Julia, Javascript, ggplot2, F# and MATLAB. We will be using the [Python](https://plotly.com/python/) library.

[Dash](https://dash.plotly.com) provides a low-code framework for rapidly building data apps in Python, R, Julia, and F#. Dash is written on top of Plotly, making it an ideal candidate for building and deploying data apps with customized user interfaces.

[Jupyter Dash](https://github.com/plotly/jupyter-dash) is a library that makes it easy to develop Plotly Dash apps from Jupyter Notebook and JupyterLab environments.

# Technical setup

We first install the Jupyter Dash library via pip. The -q option quiets the output to keep the output clear.

In [ ]:
!pip install -q jupyter-dash

Next, we import the libraries that we will use to create our interactive BI dashboard.

In [ ]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output

*Important note: when not working in a Notebook environment (.ipynb) but in a Python script (.py), simply replace the jupyter_dash.JupyterDash class with the standard dash.Dash class.*

# Read data

We will create a BI dashboard for the Ames housing data, se we first need to read our CSV data as a Pandas DataFrame.

In [ ]:
pd_ames = pd.read_csv("https://katrienantonio.github.io/hands-on-machine-learning-R-module-1/data/ames_python.csv")
pd_ames = pd_ames.drop(columns=["Unnamed: 0"])

After succesfully loading our data, we can inspect the first 10 rows with the Pandas `head` method.



In [ ]:
pd_ames.head(10)

Let's print a list of all the column names to know which info we have available.

In [ ]:
pd_ames.columns

# Interactive graphs with Plotly

We will now create some interactive data visualization graphs to show the power of Plotly. A nice overview of available graph types with examples can be found [here](https://plotly.com/python/).

## Scatterplot for continuous features

We create an interactive scatterplot for the sale price in function of the lot area, where the color and size of the points are determined by the lot shape and number of full baths respectively. We also add the house style as info to show when hovering over the points.

In [ ]:
fig_scatter = px.scatter(pd_ames, x="Lot_Area", y="Sale_Price",
                         color="Lot_Shape", size="Full_Bath", hover_data=["House_Style"],
                         opacity=0.5, log_x=False, log_y=False)
fig_scatter.show()

## Boxplot for categorical features

We create an interactive boxplot for the sale price in function of the overall house quality, ordered from low to high quality.

In [ ]:
fig_box = px.box(pd_ames, x="Overall_Qual", y="Sale_Price",
                 category_orders = {"Overall_Qual": ["Very_Poor","Poor","Fair","Below_Average","Average","Above_Average","Good","Very_Good","Excellent","Very_Excellent"]})
fig_box.show()

## Map for spatial features

We create an interactive map for the sale price in function of the latitude and longitude coordinates of the house location.

In [ ]:
fig_map = px.density_mapbox(pd_ames, lat="Latitude", lon="Longitude", z="Sale_Price", radius=5,
                            center=dict(lat=pd_ames.Latitude.mean(), lon=pd_ames.Longitude.mean()), zoom=11,
                            mapbox_style="open-street-map", hover_data=["Neighborhood"])
fig_map.show()

## Your custom graph

Now it is your turn to experiment with Plotly and create your flashy custom graph. You can find some inspiration [here](https://plotly.com/python/). Have fun!

In [ ]:
# Your custom graph

# Data Viz application with Jupyter Dash

We will now guide you through the steps to create a dashboard tool with Jupyter Dash. We will need to perform the following steps:

* Initialize the app
* Define the layout of the app
* Add functionality to the app via *callback functions*
* Run the app

## Initialize the app

We initialize the app by creating an instance object of the `jupyter_dash.JupyterDash` class.

In [ ]:
# App instance
app = JupyterDash(__name__)

What is `__name__`? This is a special variable in Python, set by the interpreter at execution time. If the source file is executed as the main program, the `__name__` variable gets the value "\_\_main\_\_". If this file is imported, `__name__` will be set to the imported module’s name.

In [ ]:
print(__name__)

## Layout of the app

We now specify the layout of our Dash app via the following HTML Elements and Dash Core Components:

* html.Div: HTML division tag which forms a container for HTML elements
* html.H1: HTML header tag to create a first-level header element
* dcc.Dropdown: Dash core component that holds a dropdown element
* dcc.Graph: Dash core component that holds a graphical element

More html elements and dash core components can be found in the Dash [documentation](https://dash.plotly.com).



In [ ]:
# Example app layout
app.layout = html.Div(children=[
                                html.H1(children='Hello Dash'),
                                html.Div(children='Choose a color for your scatterplot:\n'),
                                dcc.Dropdown(
                                    id="toy-dropdown",
                                    options=["blue","green","red","purple"],
                                    value="blue",
                                    clearable=True
                                  ),
                                dcc.Graph(
                                    id="toy-graph",
                                    figure=px.scatter(pd_ames, x="Lot_Area", y="Sale_Price")
                                  )
                                ])

## Running the app

We can now run our app via the `run_server` method while specifying a host and port number. Running in `mode = inline` makes our app appear in this notebook.

In [ ]:
# Run the app
app.run_server(mode='inline',host="0.0.0.0",port=1004)

This looks neat! However, you might have noticed that changing the dropdown color did not change the actual scatterplot color. Why not? Well we actually have not told our app what to do when we change the dropdown value.

## Functionality of the app

To add functionality into our app, we need to include callback functions. The callback function specifies the functionality to be implemented in the app, where the function decorator defines the input/output dependencies within the app via the Dash core components. You can find some examples in the Dash [documentation](https://dash.plotly.com/basic-callbacks). 

We will now add the functionality to change the graph colors via the `update_figure()` function. The decorator of the function specifies that we take the `value` field of `toy-dropdown` as an input and send the output to the `figure` field of `toy-graph`. Notice that the figure argument is no longer specified in the `dcc.Graph` as this has now become a dynamic element instead of a static scatterplot.

In [ ]:
# App instance
app = JupyterDash(__name__)

# App layout
app.layout = html.Div(children=[
                                html.H1(children='Hello Dash'),
                                html.Div(children='Choose a color for your scatterplot:\n'),
                                dcc.Dropdown(
                                    id="toy-dropdown",
                                    options=["blue","green","red","purple"],
                                    value="blue",
                                    clearable=True
                                  ),
                                dcc.Graph(
                                    id="toy-graph"
                                    # Notice that the figure argument is no longer here
                                  )
                                ])


# Define callback to update graph
@app.callback(
    Output('toy-graph', 'figure'),
    [Input("toy-dropdown", "value")]
)
def update_figure(color_value):
    return px.scatter(pd_ames, x="Lot_Area", y="Sale_Price").update_traces(marker=dict(color=color_value))


# Run the app
app.run_server(mode='inline',host="0.0.0.0",port=1004)

## An elaborate example

You now know the basic functionality of creating Dash apps. The app below create more visualizations and functionality via callbacks. Can you link the resulting app elements to the source code?

In [ ]:
# App instance
app = JupyterDash(__name__)

# App layout
app.layout = html.Div(children=[
                                # App header title
                                html.H1(children='Ames Housing Dashboard'),
                                # Left side of the app screen
                                html.Div(style={'width': '49%', 'display': 'inline-block'},
                                         children=[
                                                   # Text element
                                                   'Choose the x-name and y-scale for a numeric feature scatterplot:\n',
                                                   # Dropdown component
                                                   dcc.Dropdown(
                                                       id="num-dropdown",
                                                       options=["Lot_Area","Gr_Liv_Area","Year_Built"],
                                                       value="Lot_Area",
                                                       clearable=False
                                                       ),
                                                   # Select component
                                                   dcc.RadioItems(
                                                       id='num-scale',
                                                       options=['Linear', 'Log'],
                                                       value='Linear',
                                                       labelStyle={'display': 'inline-block'}
                                                       ),
                                                    # Graph component
                                                    dcc.Graph(
                                                        id="num-graph"
                                                        )]),
                                # Right side of the app screen
                                html.Div(style={'width': '49%', 'display': 'inline-block'},
                                         children=[
                                                   # Text element
                                                   'Choose the x-name and y-scale for a categorical feature boxplot:\n',
                                                   # Dropdown component
                                                   dcc.Dropdown(
                                                       id="cat-dropdown",
                                                       options=["Overall_Qual","Overall_Cond","MS_Zoning"],
                                                       value="Overall_Qual",
                                                       clearable=False
                                                       ),
                                                   # Select component
                                                   dcc.RadioItems(
                                                       id='cat-scale',
                                                       options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                                                       value='Linear',
                                                       labelStyle={'display': 'inline-block'}
                                                       ),
                                                    # Graph component
                                                    dcc.Graph(
                                                        id="cat-graph"
                                                        )])
                                
                               
                                ])

# Callback function for numeric scatterplot
@app.callback(
    Output('num-graph', 'figure'),
    [Input('num-dropdown', 'value'),
     Input('num-scale', 'value')])
def update_num_graph(num_feature, num_scale):
  return px.scatter(pd_ames, x=num_feature, y="Sale_Price", opacity=0.5, log_x=False, log_y=(num_scale=='Log'))

# Callback function for catergorical boxplot
@app.callback(
    Output('cat-graph', 'figure'),
    [Input('cat-dropdown', 'value'),
     Input('cat-scale', 'value')])
def update_cat_graph(cat_feature, cat_scale):
  return px.box(pd_ames, x=cat_feature, y="Sale_Price", log_x=False, log_y=(cat_scale=='Log'))
    

# Run the app
app.run_server(mode='inline',host="0.0.0.0",port=1004)

## Your custom app

Now it's your time. Feel free to experiment by building on the example app or by creating a new Dash app from scratch. Have fun!

In [ ]:
# Your custom Dash app